In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
acs_dataframe_raw = pd.read_csv('Datathon_Data/acs_5yr_est_selected_economic_characteristics_2010-2022.csv')
meat_production_raw = pd.read_csv('Datathon_Data/Meat_Stats_Meat_Production.csv')
nutrition_raw = pd.read_csv('Datathon_Data/Nutrition_Physical_Activity_and_Obesity_Data.csv')
acs_dataframe = acs_dataframe_raw.copy()
meat_production = meat_production_raw.copy()
nutrition = nutrition_raw.copy()
meat_production['Production'] = meat_production['Production'].str.replace(",","").astype(float)
nutrition.columns

/var/folders/96/s7zm7_qd5tl766bx8kyp31gm0000gn/T/ipykernel_68267/175944214.py:3: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrition_raw = pd.read_csv('Datathon_Data/Nutrition_Physical_Activity_and_Obesity_Data.csv')


Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'Class', 'Topic', 'Question', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Alt', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Low_Confidence_Limit', 'High_Confidence_Limit',
       'Sample_Size', 'Total', 'Age(years)', 'Education', 'Gender', 'Grade',
       'Income', 'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID',
       'QuestionID', 'DataValueTypeID', 'LocationID',
       'StratificationCategory1', 'Stratification1',
       'StratificationCategoryId1', 'StratificationID1'],
      dtype='object')

In [3]:
acs_dataframe['Label (Grouping)'] = acs_dataframe['Label (Grouping)'].str.strip()
employed = acs_dataframe[(acs_dataframe['Label (Grouping)'] == 'Employed')].copy()
employed['Estimate'] = employed['Estimate'].str.replace(",", "").astype(int)
public_coverage = acs_dataframe[(acs_dataframe['Label (Grouping)'] == 'With public coverage') 
                                & (acs_dataframe['Estimate'] != '(X)')].copy()
public_coverage['Estimate'] = public_coverage['Estimate'].str.replace(",", "").astype(float)

public_coverage.head(10)

,Label (Grouping),Category,State,Estimate,Margin of Error,Percent,Percent Margin of Error,Year
15184,With public coverage,HEALTH INSURANCE COVERAGE,Alabama,1568842.0,"±7,998",33.3%,±0.2,2013
15193,With public coverage,HEALTH INSURANCE COVERAGE,Alabama,99342.0,"±2,608",5.2%,±0.1,2013
15198,With public coverage,HEALTH INSURANCE COVERAGE,Alabama,36190.0,"±1,467",16.0%,±0.6,2013
15203,With public coverage,HEALTH INSURANCE COVERAGE,Alabama,316315.0,"±3,991",38.8%,±0.4,2013
15329,With public coverage,HEALTH INSURANCE COVERAGE,Alaska,173846.0,"±2,857",24.9%,±0.4,2013
15338,With public coverage,HEALTH INSURANCE COVERAGE,Alaska,24642.0,"±1,086",7.6%,±0.3,2013
15343,With public coverage,HEALTH INSURANCE COVERAGE,Alaska,6095.0,±464,19.8%,±1.3,2013
15348,With public coverage,HEALTH INSURANCE COVERAGE,Alaska,26710.0,"±1,035",27.9%,±0.9,2013
15474,With public coverage,HEALTH INSURANCE COVERAGE,Arizona,2163069.0,"±12,230",33.9%,±0.2,2013
15483,With public coverage,HEALTH INSURANCE COVERAGE,Arizona,237598.0,"±3,588",9.2%,±0.1,2013


In [4]:
yearly_df = pd.DataFrame(columns=['Year', 'Employed', 'Meat_Production', 'Public_Coverage', 'Obesity'])
for year in range(2012, 2023):
    total_employed = (employed[employed['Year'] == year]['Estimate'].sum())
    total_meat_production = meat_production[meat_production['Year'] == year]['Production'].sum()
    total_public_coverage = public_coverage[public_coverage['Year'] == year]['Estimate'].sum()
    mean_obesity_us = nutrition[(nutrition['YearEnd'] == year) & 
                                (nutrition['Question'] == 'Percent of adults aged 18 years and older who have obesity')]['Data_Value'].mean()
    yearly_df = pd.concat([yearly_df, pd.DataFrame({'Year': [year], 
                                                    'Employed': [total_employed], 
                                                    'Meat_Production': [total_meat_production],
                                                    'Public_Coverage': [total_public_coverage],
                                                    'Obesity' : mean_obesity_us})], ignore_index=True)
    
yearly_df['Employed'] = pd.to_numeric(yearly_df['Employed'], errors="coerce")
print(yearly_df.dtypes)
corr_mat = yearly_df.corr()
print(corr_mat)


/var/folders/96/s7zm7_qd5tl766bx8kyp31gm0000gn/T/ipykernel_68267/1339112690.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  yearly_df = pd.concat([yearly_df, pd.DataFrame({'Year': [year],


Year                object
Employed             int64
Meat_Production    float64
Public_Coverage    float64
Obesity            float64
dtype: object
                     Year  Employed  Meat_Production  Public_Coverage  \
Year             1.000000  0.991997         0.957357         0.982592   
Employed         0.991997  1.000000         0.981279         0.981897   
Meat_Production  0.957357  0.981279         1.000000         0.943883   
Public_Coverage  0.982592  0.981897         0.943883         1.000000   
Obesity          0.990776  0.981622         0.949928         0.962757   

                  Obesity  
Year             0.990776  
Employed         0.981622  
Meat_Production  0.949928  
Public_Coverage  0.962757  
Obesity          1.000000  


### Now we aim to build a robust regression model that could be potentially used to predict future obesity rate in the United States.

In [5]:
yearly_df = sm.add_constant(yearly_df)
X = yearly_df[['Employed', 'Meat_Production', 'Public_Coverage']]
Y = yearly_df['Obesity']
model = sm.OLS(Y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                Obesity   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.309e+04
Date:                Wed, 31 Jul 2024   Prob (F-statistic):                    4.24e-15
Time:                        13:23:14   Log-Likelihood:                         -6.6719
No. Observations:                  11   AIC:                                      19.34
Df Residuals:                       8   BIC:                                      20.54
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

/Users/tony/myenv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=11 observations were given.
  return hypotest_fun_in(*args, **kwds)
